In [5]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

import animal_shelter


# Importing AnimalShelter class 
from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
username = "aacUser"
password = "animals"
animals = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(animals.read({}))


#########################
# Dashboard Layout / View
#########################
# create dash application

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)


image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
                               
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Jacob Silberstein Project 2 Dashboard'))),
    html.Hr(),
    html.Center(html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image')),
    html.Div(      
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue unit', 'value': 'water'},
                {'label': 'Wilderness Exploration and Recovery Unit', 'value': 'Wild'},
                {'label': 'Disaster Relief and Individual Tracking Unit', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        )
        
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # Starts on page 0 and allows up to 10 rows.
        #Columns and Rows may be selected but not deleted/edited
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=True,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



# Radio button filters
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
        # Water Rescue filter
        if filter_type == 'water':
            df = pd.DataFrame.from_records(animals.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"
                                 ]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
        # Wilderness Exploration and Recovery Unit Filter
        elif filter_type == 'Wild':
            df = pd.DataFrame.from_records(animals.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
        # Disaster Relief and Individual Tracking Unit Filter
        elif filter_type == 'disaster':
            df = pd.DataFrame.from_records(animals.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", 
                                  "Bloodhound","Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            }))
            
        # Reset to no filter
        else:
            df = pd.DataFrame.from_records(animals.read({}))
        #Restores to original settings as of opening Dash
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)

# Change background color of selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#b7e7da'
    } for i in selected_columns]

# Histogram chart for showing amount of animals available
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    ])
def update_graphs(viewData):
    ###FIXED ####
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(
            figure = px.histogram(dff, x='breed')
        )    
    ]

# Geolocation map for idenitfying available animals
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'derived_viewport_selected_rows')])
def update_map(viewData, row_ids):
    dff = pd.DataFrame.from_dict(viewData)
    # Sets first num in row_ids array to the last selected row ID
    row_ids[0] = row_ids[len(row_ids) - 1]
    return [
        # row_ids[0] gets last selected row from DataTable and parameters 13 and 14 are long and lat of rows
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], children=[
                # parameter num 4 gives the breed of animal
                dl.Tooltip(dff.iloc[row_ids[0],4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    # parameter 9 gives animal's name
                    html.P(dff.iloc[row_ids[0],9])
                ])
            ])
        ])
    ]


if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Apr/2023 22:20:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 22:20:21] "GET /_dash-component-suites/dash_renderer/react@16.v1_3_0m1588250875.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 22:20:21] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_3_0m1588250875.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 22:20:21] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_3_0m1588250875.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 22:20:21] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_3_0m1588250875.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 22:20:21] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_9_0m1588250878.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 22:20:21] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1_9_0m1588250878.min.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 22:

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.6/site-packages/dash/dash.py", line 1

127.0.0.1 - - [20/Apr/2023 22:20:22] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.6/site-packages/dash/dash.py", line 1

127.0.0.1 - - [20/Apr/2023 22:20:22] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.6/site-packages/dash/dash.py", line 1

127.0.0.1 - - [20/Apr/2023 22:20:22] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.6/site-packages/dash/dash.py", line 1

127.0.0.1 - - [20/Apr/2023 22:20:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 22:20:22] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [20/Apr/2023 22:20:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 22:20:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 22:20:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 22:41:10] "POST /_dash-update-component HTTP/1.1" 200 -
